In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sys
sys.path.append('../src')
import escrapeos as esc
import json


# Escrapeo Nombre trastronos CIE-10

[wiki](https://es.wikipedia.org/wiki/Anexo:CIE-10_Cap%C3%ADtulo_V:_Trastornos_mentales_y_del_comportamiento)

In [2]:
url = "https://es.wikipedia.org/wiki/Anexo:CIE-10_Cap%C3%ADtulo_V:_Trastornos_mentales_y_del_comportamiento"
elemento = 'h2'
# html = requests.get(url)
# soup = BeautifulSoup(html.content,"html.parser")

In [3]:
capitulo = esc.capitulos(url,elemento)

In [4]:
capitulo.sample(3)

,cap,title
4,F40-49,"Trastornos neuróticos, trastornos relacionados..."
9,F90-F98,Trastornos emocionales y del comportamiento qu...
0,F00-F09,"Trastornos mentales orgánicos, incluidos los t..."


In [5]:
elementodos = 'li'

In [6]:
diagnostic = esc.diagnostics(url,elementodos)

In [7]:
diagnostic.sample(3)

,code,diag
203,(F64.0),Transsexualismo
94,F31.32,"Trastorno bipolar, episodio actual depresivo, ..."
192,(F60.6),Trastorno ansioso o por evitación de la person...


### limpieaza dataframe diag

In [8]:
sin_p = {r"[(]":"",r"[)]":""}#quitamos los paréntesis 
diagnostic.code = diagnostic.code.replace(sin_p,regex=True) 


In [9]:
diagnostic.sample(3)

,code,diag
117,F32,"Trastorno depresivo mayor, episodio único\nF32..."
223,F68.1,Producción intencionada o ficción de síntomas ...
248,F84,Trastorno generalizado del desarrollo\n(F84.0)...


In [10]:
falta = {'code':'F65','diag':'Trastornos de la preferencia sexual'}

In [11]:
diagnostic = diagnostic.append(falta, ignore_index=True)

In [12]:
diagnostic.tail(3)

,code,diag
287,F98.6,Desorden lingüístico
288,F99,Trastorno mental no especificado en otra parte
289,F65,Trastornos de la preferencia sexual


In [13]:
#nos quedamos solo con los subcodigos que sean principales (F00-F99)
diagnostic = diagnostic[diagnostic["code"].str.match(r"(^F..$)")]
len(diagnostic)

77

In [14]:
for i,r in diagnostic.iterrows():
    if "\n" in r.diag:
        r.diag = r.diag.split("\n")[0]

In [15]:
diagnostic.sample(3)

,code,diag
150,F43,Reacción al stress grave y trastornos de adapt...
38,F15,Trastornos mentales y de comportamiento debido...
35,F12,Trastornos mentales y de comportamiento debido...


In [16]:
#añadimos el capítulo correpondiente a cada diagnóstico de diagnostic

In [17]:
correspond = {'F00-F09': ["F00","F01","F02","F03","F04","F05","F06","F07","F08","F09"],
'F10-F19': ["F10","F11","F12","F13","F14","F15","F16","F17","F18","F19"],
'F20-29' : ["F20","F21","F22","F23","F24","F25","F26","F27","F28","F29"],
'F30-39' : ["F30","F31","F32","F33","F34","F35","F36","F37","F38","F39"],
'F40-49': ["F40","F41","F42","F43","F44","F45","F46","F47","F48","F49"],
'F50-59': ["F50","F51","F52","F53","F54","F55","F56","F57","F58","F59"],
'F60-69': ["F60","F61","F62","F63","F64","F65","F66","F67","F68","F69"],
'F70-79':["F70","F71","F72","F73","F74","F75","F76","F77","F78","F79"],
'F80-89' : ["F80","F81","F82","F83","F84","F85","F86","F87","F88","F89"],
'F90-F98':["F90","F91","F92","F93","F94","F95","F96","F97","F98"],
'F99':["F99"]}

In [18]:
dicc_corr = {}
for key,value in correspond.items():
    for i in range(len(value)):
        dicc_corr[value[i]] = key

In [19]:
diagnostic['cap']=diagnostic.code.map(dicc_corr)

In [20]:
diagnostic = diagnostic.merge(capitulo, how='right')

In [21]:
diagnostic.sample(10)

,code,diag,cap,title
12,F12,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...
57,F71,Retraso mental moderado,F70-79,Retraso mental
58,F72,Retraso mental severo,F70-79,Retraso mental
71,F92,Trastornos mixtos de conducta y emociones,F90-F98,Trastornos emocionales y del comportamiento qu...
32,F34,Trastornos persistentes del estado de ánimo [a...,F30-39,"Trastornos del estado de animo, trastornos afe..."
33,F39,"Trastorno del estado de ánimo [afectivo], no e...",F30-39,"Trastornos del estado de animo, trastornos afe..."
61,F79,Retrasos mentales sin especificar,F70-79,Retraso mental
18,F18,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...
35,F41,Otros trastornos de ansiedad,F40-49,"Trastornos neuróticos, trastornos relacionados..."
11,F11,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...


In [22]:
resumen = {'Trastornos mentales orgánicos, incluidos los trastornos sintomáticos': 'orgánicos/sintomáticos' ,
       'Trastornos mentales y de comportamiento debidos al consumo de psicotrópicos': 'consumo psicotrópicos',
       'Esquizofrenia, trastornos esquizotípicos y trastornos delirantes': 'esquizotípicos/delirantes',
       'Trastornos del estado de animo, trastornos afectivos':'ánimo/afectivos',
       'Trastornos neuróticos, trastornos relacionados con el estrés y trastornos somatomorfos':'neuróticos/somatomorfos',
       'Síndromes del comportamiento asociados con alteraciones fisiológicas y factores físicos':'factores físicos /fisiológicos',
       'Trastornos de la personalidad y del comportamiento en adultos':'T. de la personalidad/comportamiento',
       'Retraso mental':'deficiencia psíquica', 'Trastornos del desarrollo psicológico':'desarrollo psicológico',
       'Trastornos emocionales y del comportamiento que aparecen habitualmente en la niñez o en la adolescencia':'aparecen en niñez',
       'Trastornos mentales sin especificar':'sin espcificar'}

In [23]:
diagnostic['resumen_cap'] = diagnostic.title.map(resumen)

In [24]:
diagnostic.to_csv("../data/diagnos.csv", index=False)

#  Fases lunares

In [25]:
url_ = 'https://calendarios.ideal.es/calendario-lunar/fases-lunares/'

In [26]:
lunas = esc.calendariolunar(url_,2016,2020)

/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'calendarios.ideal.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'calendarios.ideal.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/sinatxe/anaconda3/envs/ironhack/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'calendarios.ideal.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-

In [27]:
lunas.sample(3)

,fecha,moon,fecha_Date
113,15/04/2018,Luna Nueva,2018-04-15
180,22/08/2019,Cuarto Menguante,2019-08-22
168,26/05/2019,Cuarto Menguante,2019-05-26


In [28]:
lunas.to_csv('../data/cal_lunar.csv',index=False)

# Horas de luz

[web](https://www.epdata.es/horas-luz-provincias-2019/6841c77e-e946-420e-a0c4-cff4de1154f3#)

In [29]:
luz = pd.read_csv('../data/ambientales/horas_de_luz_por_provincias.csv',sep=';')
luz = luz[0:624].drop('Año',axis=1).rename({'Periodo':'mes','Parámetro':'Provincia','Salida y puesta del sol':'horas'},axis=1)
luz.horas = luz.horas.str.replace(',','.').astype('float')
luz.Provincia = luz.Provincia.str.strip()
luz.Provincia = luz.Provincia.str.replace('Islas Baleares','Baleares')

In [30]:
luz.sample(3)

,mes,Provincia,horas
36,Enero,Salamanca,9.62
268,Junio,Burgos,15.23
602,Diciembre,Navarra,9.09


In [31]:
#coordenar nombre de provincias con geojson provincial
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

prov_cod = {}
for i in range(52):    
    prov_cod[provincias['features'][i]['properties']['name']] = provincias['features'][i]['properties']['cod_prov']

In [32]:
luz.Provincia.unique()

array(['Álava', 'Albacete', 'Alicante', 'Almería', 'Ávila', 'Badajoz',
       'Baleares', 'Barcelona', 'Burgos', 'Cáceres', 'Cádiz', 'Castellón',
       'Ciudad Real', 'Córdoba', 'A Coruña', 'Cuenca', 'Girona',
       'Granada', 'Guadalajara', 'Guipúzcoa', 'Huelva', 'Huesca', 'Jaén',
       'León', 'Lleida', 'La Rioja', 'Lugo', 'Madrid', 'Málaga', 'Murcia',
       'Navarra', 'Ourense', 'Asturias', 'Palencia', 'Las Palmas',
       'Pontevedra', 'Salamanca', 'Santa Cruz de Tenerife', 'Cantabria',
       'Segovia', 'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo',
       'Valencia', 'Valladolid', 'Vizcaya', 'Zamora', 'Zaragoza', 'Ceuta',
       'Melilla'], dtype=object)

In [33]:
luz['cod'] = luz.Provincia.map(prov_cod)

In [34]:
mes_num = {'Enero':1, 'Febrero':2, 'Marzo':3, 'Abril':4, 'Mayo':5, 'Junio':6, 'Julio':7,
       'Agosto':8, 'Septiembre':9, 'Octubre':10, 'Noviembre':11, 'Diciembre':12}

In [35]:
luz['mes_n'] = luz.mes.map(mes_num)

In [36]:
luz.sample(3)

,mes,Provincia,horas,cod,mes_n
56,Febrero,Ávila,10.66,05,2
214,Mayo,Baleares,14.35,07,5
617,Diciembre,Valencia,9.44,46,12


In [37]:
luz.to_csv('../data/luzmesespr.csv',index=False)

# Provincias

In [38]:
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

In [39]:
cod_prov = {}
for i in range(52):    
    cod_prov[provincias['features'][i]['properties']['cod_prov']] = provincias['features'][i]['properties']['name']

In [40]:
provincias_cod = pd.DataFrame([[key, cod_prov[key]] for key in cod_prov.keys()], columns=['Cod', 'Provincia'])

In [41]:
provincias_cod.to_csv('../data/codigos_provincia.csv',index=False)